In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

In [15]:
%run ../scripts/post_scraping_text_processing.py functions

functions loaded


In [7]:
train = pd.read_csv('../assets/train.csv', index_col=0)
test = pd.read_csv('../assets/test.csv', index_col = 0)
df = pd.concat([train, test], axis = 0)

In [9]:
print(train.shape)
print(test.shape)
print(df.shape)

(11568, 5)
(3857, 5)
(15425, 5)


In [10]:
precoup = df[df['written_precoup']==1]
postcoup = df[df['written_precoup'] ==0]

In [21]:
precoup_corpus = precoup['text'].copy()
postcoup_corpus = postcoup['text'].copy()

In [19]:
cvec_pre = CountVectorizer(stop_words=STOP_WORDS, min_df=2)

In [20]:
cvec_pre.fit(precoup_corpus)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None,
        stop_words={'keşke', 'yakında', 'büsbütün', 'kelli', 'berisi', 'yakınlarda', 'illa', 'yaptıkları', 'bilcümle', 'lütfen', 'evvelemirde', 'öbürkü', 'tamam', 'gibilerden', 'olursa', 'birçoğunda', 'tek', 'tamamıyla', 'çoğunun', 'zati', 'bizzat', 'halihazırda', 'kendisine', 'bazen', 'biri', 'değin', 'kim...birazdan', 'hulasaten', 'hiç', 'elbet', 'çoğunca', 'başkasının', 'herkesi', 'nedenle', 'ki', 'defa'},
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [23]:
transformed = cvec_pre.transform(precoup_corpus)

In [32]:
pd.DataFrame(pd.DataFrame(transformed.toarray(), columns=cvec_pre.get_feature_names()).sum().sort_values(ascending =False), columns = ['count'])

,count
bir,6569
banah,1605
gel,1460
sanah,1429
yar,1212
ah,1073
dah,1062
aman,989
gün,870
aşk,717


In [33]:
def make_vocab_list(corpus, stopwords = None, ngram_range = (1,1), min_df = 1):
    """Takes a corpus and returns a one column dataframe with words in the index and counts in a column.
    
    Options:
        - stopwords: either a list-like or None (default)
        - ngram_range: a tuple (default (1,1))
        - min_df: minimum document frequency (default 1)"""
    cv = CountVectorizer(stop_words=stopwords, ngram_range=ngram_range, min_df = min_df)
    cv.fit(corpus)
    transformed = cv.transform(corpus)
    
    return pd.DataFrame(pd.DataFrame(transformed.toarray(), columns=cv.get_feature_names()).sum().sort_values(ascending =False), columns = ['count'])
    
    

In [40]:
words_precoup = make_vocab_list(precoup_corpus)

words_postcoup = make_vocab_list(postcoup_corpus)

In [41]:
total_precoup = words_precoup['count'].sum()
print(total_precoup)

total_postcoup = words_postcoup['count'].sum()
print(total_postcoup)

280946
751741


In [42]:
words_precoup['proportion'] = words_precoup['count']/total_precoup
words_postcoup['proportion'] = words_postcoup['count']/total_postcoup

In [46]:
words_precoup.head(30)

,count,proportion
bir,6569,0.023382
bu,3485,0.012405
ne,2935,0.010447
ben,2827,0.010062
beni,2647,0.009422
sen,2409,0.008575
seni,2096,0.007461
banah,1605,0.005713
gibi,1560,0.005553
de,1552,0.005524


In [45]:
words_postcoup.head()

,count,proportion
bir,16505,0.021956
bu,9736,0.012951
ben,8013,0.010659
ne,6864,0.009131
sen,6640,0.008833


In [49]:
words_precoup.merge(words_postcoup, left_index=True, right_index=True)

,count_x,proportion_x,count_y,proportion_y
bir,6569,0.023382,16505,0.021956
bu,3485,0.012405,9736,0.012951
ne,2935,0.010447,6864,0.009131
ben,2827,0.010062,8013,0.010659
beni,2647,0.009422,6409,0.008526
sen,2409,0.008575,6640,0.008833
seni,2096,0.007461,4894,0.006510
banah,1605,0.005713,4082,0.005430
gibi,1560,0.005553,4036,0.005369
de,1552,0.005524,4159,0.005532


In [51]:
print(len(df['text'].unique()))

15176


In [52]:
df.shape

(15425, 5)

In [54]:
df['text'].value_counts()

Şarkı enstrümantal olduğu için şarkı sözü bulunmamaktadır                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [55]:
df[df['text'] == "Dereler taştı geldi dereler taştı geldi Dağları aştı geldi yar Fadime dağları aştı geldi Ben gece bekliyordum ben gece bekliyordum O gündüz kaçtı geldi yar Fadime o gündüz kaçtı geldi Dereden geçiyorken dereden geçiyorken Atladım taştan taşah yar Fadime atladım taştan taşah Böyle sevdalıkları böyle sevdalıkları Mevlam vermesin başah yar Fadime Allah vermesin başah İn dereye dereye in dereye dereye Dere alanlarınah yar Fadime dere alanlarınah Bakmayasın ufağım bakmayasın ufağım Elin yalanlarınah yar Fadime elin yalanlarınah İn dereye dereye in dereye dereye Hah ben de geliyorum yar Fadime hem ben de geliyorum Üç görmedim seni üç gün görmedim seni Derdimden ölüyorum yar Fadime derdimden ölüyorum"]

,artist,album,text,name,written_precoup
28988,İsmail Türüt,Sosyete Kızı Suzan,Dereler taştı geldi dereler taştı geldi Dağlar...,Yar Yar,1
28987,İsmail Türüt,Piryoz,Dereler taştı geldi dereler taştı geldi Dağlar...,Yar Fadimem,0


In [57]:
df[df['text'] == "Başın öne eğilmesin Aldırmah gönül aldırmah Ağladığın duyulmasın Aldırmah gönül aldırmah Dışarıdah deli dalgalar Gelip duvarları yalar Seni bu sesler oyalar Aldırmah gönül aldırmah Kurşun atah atah biter Yollar gide gide biter Mapus yatah yatah biter Aldırmah gönül aldırmah Dertlerin kalkıncah şahah Bir sitem yollah Allahah Görecek günler var dahah Aldırmah gönül aldırma"]

,artist,album,text,name,written_precoup
19382,Edip Akbayram,Dün Ve Bugün,Başın öne eğilmesin Aldırmah gönül aldırmah Ağ...,Aldırma Gönül,1
27144,Hayko Cepkin,Beni Büyüten Şarkılar vol. 1,Başın öne eğilmesin Aldırmah gönül aldırmah Ağ...,Aldırma Gönül,0
3039,Ankaralı Namık,Dar Geldi Sana Ankara,Başın öne eğilmesin Aldırmah gönül aldırmah Ağ...,Aldırma Gönül,0
1374,Ahmet Özhan,Geceler Gariplerindir,Başın öne eğilmesin Aldırmah gönül aldırmah Ağ...,Aldırma Gönül,1
